In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
%%writefile example.cu
#include <stdio.h>

__global__ void helloFromGPU() {
    printf("Hello World from GPU!\n");
}

int main() {
    helloFromGPU<<<1, 10>>>();
    cudaDeviceSynchronize();
    return 0;
}


Writing example.cu


In [ ]:
!nvcc -o example example.cu
!./example

Hello World from GPU!
Hello World from GPU!
Hello World from GPU!
Hello World from GPU!
Hello World from GPU!
Hello World from GPU!
Hello World from GPU!
Hello World from GPU!
Hello World from GPU!
Hello World from GPU!


In [ ]:
%%writefile Memory_Allocation.cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <stdio.h>

void checkDeviceMemory(void)
{
  size_t free, total;
  cudaMemGetInfo(&free, &total);
  printf("Device memory (free/total) = %lld/%lld bytes\n", free, total);
}

int main() {
  int* dDataPtr;
  cudaError_t errorCode;

  checkDeviceMemory();
  errorCode = cudaMalloc(&dDataPtr, sizeof(int) * 1024 * 1024); // 4MB 할당
  printf("cudaMalloc - %s\n", cudaGetErrorName(errorCode));
  checkDeviceMemory();

  errorCode = cudaMemset(dDataPtr, 0, sizeof(int) * 1024 * 1024); // 모두 0으로 초기화
  printf("cudaMemset - %s\n", cudaGetErrorName(errorCode));

  errorCode = cudaFree(dDataPtr); // 할당된 메모리 공간 해제
  printf("cudaFree - %s\n", cudaGetErrorName(errorCode));
    return 0;
}


Writing Memory_Allocation.cu


In [ ]:
!nvcc -o Memory_Allocation Memory_Allocation.cu
!./Memory_Allocation

Memory_Allocation.cu: In function ‘void checkDeviceMemory()’:
Memory_Allocation.cu:9:8: warning: format ‘%lld’ expects argument of type ‘long long int’, but argument 2 has type ‘size_t’ {aka ‘long unsigned int’} []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat=-Wformat=]8;;]
    9 |   printf("Device memory (free/total) = %lld/%lld bytes\n", free, total);
      |        ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  ~~~~
      |                                                          |
      |                                                          size_t {aka long unsigned int}
Memory_Allocation.cu:9:8: warning: format ‘%lld’ expects argument of type ‘long long int’, but argument 3 has type ‘size_t’ {aka ‘long unsigned int’} []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat=-Wformat=]8;;]
    9 |   printf("Device memory (free/total) = %lld/%lld bytes\n", free, total);
      |        ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [ ]:
%%writefile Memcpy.cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <stdio.h>

__global__ void printData(int* _dDataPtr) {
  printf("%d", _dDataPtr[threadIdx.x]);
} // printData 커널(스레드별 자신이 담당하는 원소의 번호가 threadIdx.x가 됨)


__global__ void setData(int* _dDataPtr) {
  _dDataPtr[threadIdx.x] = 2;
}

int main() {
    int data[10] = {0};
    for (int i = 0; i < 10; i++) data[i] = 1;

    int* dDataPtr;
    cudaMalloc(&dDataPtr, sizeof(int) * 10); // 40byte
    cudaMemset(dDataPtr, 0, sizeof(int) * 10); // 0으로 초기화

    printf("Data in device: ");
    printData <<<1, 10>>> (dDataPtr);

    cudaMemcpy(dDataPtr, data, sizeof(int) * 10, cudaMemcpyHostToDevice);
    // data(host)로부터 dDataPtr(device)로 40byte만큼 옮김
    printf("\nHost -> Device: ");
    printData <<<1, 10>>> (dDataPtr); // 1로 변화된 데이터

    setData <<<1, 10>>> (dDataPtr); // 2로 세팅

    cudaMemcpy(data, dDataPtr, sizeof(int)*10, cudaMemcpyDeviceToHost);
    // gpu에서 cpu로 데이터를 옮김
    printf("\nDevice-> Host: ");
    for (int i = 0; i < 10; i++) printf("%d", data[i]);

    cudaFree(dDataPtr);
    return 0;
}


Overwriting Memcpy.cu


In [ ]:
!nvcc -o Memcpy Memcpy.cu
!./Memcpy

Data in device: 0000000000
Host -> Device: 1111111111
Device-> Host: 2222222222

In [ ]:
%%writefile Vector_Sum.cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <iostream>
#include <chrono>

// #define NUM_DATA 1024
#define NUM_DATA 134217728

// Simple vector sum kernel
__global__ void vecAdd(int* _a, int* _b, int* _c, int _size) {
    int tID = blockIdx.x * blockDim.x + threadIdx.x;
    if (tID < _size) _c[tID] = _a[tID] + _b[tID];
    // 최대 블럭 스레드 번호는 tID 계산에서 고려되지 않음
}

int main() {

    int *a, *b, *c, *hc; // Vectors on host
    int *da, *db, *dc; // Vectors on device

    int memSize = sizeof(int) * NUM_DATA;
    printf("%d elements, memSize = %d bytes\n", NUM_DATA, memSize);

    // memory-allocation on the host-side
    a = new int[NUM_DATA]; memset(a, 0, memSize);
    b = new int[NUM_DATA]; memset(b, 0, memSize);
    c = new int[NUM_DATA]; memset(c, 0, memSize);
    hc = new int[NUM_DATA]; memset(hc, 0, memSize);

    // Data generation
    for (int i = 0; i < NUM_DATA; i++) {
        a[i] = rand() % 10;
        b[i] = rand() % 10;
    }

    auto start_host = std::chrono::high_resolution_clock::now();

    // vector sum on host (for performance comparision)
    for (int i = 0; i < NUM_DATA; i++) {
        hc[i] = a[i] + b[i];
    }

    auto end_host = std::chrono::high_resolution_clock::now();

    std::chrono::duration<double> elapsed_host = end_host - start_host;
    std::cout << "Host Elapsed time: " << elapsed_host.count() << " seconds" << std::endl;

    // Memory allocation on device
    cudaMalloc(&da, memSize); cudaMemset(da, 0, memSize);
    cudaMalloc(&db, memSize); cudaMemset(db, 0, memSize);
    cudaMalloc(&dc, memSize); cudaMemset(dc, 0, memSize);

    // Data Copy : Host -> Device
    cudaMemcpy(da, a, memSize, cudaMemcpyHostToDevice);
    cudaMemcpy(db, b, memSize, cudaMemcpyHostToDevice);

    auto start_device = std::chrono::high_resolution_clock::now();

    // kernel call
    // vecAdd <<<1, NUM_DATA>>> (da, db, dc); // NUM_DATA가 1024이하일때만 thread block 생성 가능

    dim3 dimGrid(ceil((float)NUM_DATA / 256), 1, 1);
    dim3 dimBlock(256, 1, 1);
    vecAdd <<< dimGrid, dimBlock >>> (da, db, dc, NUM_DATA);
    cudaDeviceSynchronize(); // 디바이스가 수행중인 작업이 끝날 때까지 대기

    auto end_device = std::chrono::high_resolution_clock::now();

    std::chrono::duration<double> elapsed_device = end_device - start_device;
    std::cout << "Device Elapsed time: " << elapsed_device.count() << " seconds" << std::endl;

    //Copy results: Device -> Host
    cudaMemcpy(c, dc, memSize, cudaMemcpyDeviceToHost);

    // Release device memory
    cudaFree(da); cudaFree(db); cudaFree(dc);

    // check results
    bool result = true;
    for (int i = 0; i < NUM_DATA; i++) {
        if (hc[i] != c[i]) {
            printf("[%d] The result is not matched! (%d, %d) \n"
            , i, hc[i], c[i]);
            result = false;
        }
    }

    if(result) printf("GPU works well!\n");

    // Release host memory
    delete[] a; delete[] b; delete[] c;

    return 0;
}


Overwriting Vector_Sum.cu


In [ ]:
!nvcc -o Vector_Sum Vector_Sum.cu
!./Vector_Sum

134217728 elements, memSize = 536870912 bytes
Host Elapsed time: 0.497378 seconds
Device Elapsed time: 0.0064109 seconds
GPU works well!


In [ ]:
%%writefile thread_layout.cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

__global__ void checkIndex(void) {
    printf("threadIdx:(%d, %d, %d) blockIdx: (%d, %d, %d) blockDim:(%d, %d, %d), gridDim: (%d, %d, %d)\n",
    threadIdx.x, threadIdx.y, threadIdx.z,
    blockIdx.x, blockIdx.y, blockIdx.z,
    blockDim.x, blockDim.y, blockDim.z,
    gridDim.x, gridDim.y, gridDim.z);
}

int main() {

    dim3 dimBlock(3, 1, 1); // or dimBlock(3)
    dim3 dimGrid(2, 1, 1); // or dimGrid(2)

    printf("dimGrid.x=%d dimGrid.y=%d dimGrid.z=%d\n", dimGrid.x, dimGrid.y, dimGrid.z);
    printf("dimBlock.x=%d dimBlock.y=%d dimBlock.z=%d\n", dimBlock.x, dimBlock.y, dimBlock.z);

    checkIndex<<<dimGrid, dimBlock>>> ();

    // GPU가 커널 실행을 완료할 때까지 대기
    cudaDeviceSynchronize();

    return 0;
}

Writing thread_layout.cu


In [ ]:
!nvcc -o thread_layout thread_layout.cu
!./thread_layout

dimGrid.x=2 dimGrid.y=1 dimGrid.z=1
dimBlock.x=3 dimBlock.y=1 dimBlock.z=1
threadIdx:(0, 0, 0) blockIdx: (0, 0, 0) blockDim:(3, 1, 1), gridDim: (2, 1, 1)
threadIdx:(1, 0, 0) blockIdx: (0, 0, 0) blockDim:(3, 1, 1), gridDim: (2, 1, 1)
threadIdx:(2, 0, 0) blockIdx: (0, 0, 0) blockDim:(3, 1, 1), gridDim: (2, 1, 1)
threadIdx:(0, 0, 0) blockIdx: (1, 0, 0) blockDim:(3, 1, 1), gridDim: (2, 1, 1)
threadIdx:(1, 0, 0) blockIdx: (1, 0, 0) blockDim:(3, 1, 1), gridDim: (2, 1, 1)
threadIdx:(2, 0, 0) blockIdx: (1, 0, 0) blockDim:(3, 1, 1), gridDim: (2, 1, 1)


In [ ]:
%%writefile CUDA_definitions.cuh
#ifndef CUDA_DEFINITIONS_CUH
#define CUDA_DEFINITIONS_CUH

// BLOCK ID
#define BID_X blockIdx.x
#define BID_Y blockIdx.y
#define BID_Z blockIdx.z

// Thread ID
#define TID_X threadIdx.x
#define TID_Y threadIdx.y
#define TID_Z threadIdx.z

// Dimension of a block
#define Bdim_X blockDim.x
#define Bdim_Y blockDim.y
#define Bdim_Z blockDim.z

// Dimension of a grid
#define Gdim_X gridDim.x
#define Gdim_Y gridDim.y
#define Gdim_Z gridDim.z

// global thread ID in blocks
#define TID_IN_BLOCK (TID_Z*(Bdim_Y*Bdim_X) + TID_Y*Bdim_X + TID_X)

// number of threads in a block
#define NUM_THREAD_IN_BLOCK (Bdim_X*Bdim_Y*Bdim_Z)

// global thread ID in grids
#define GRID_1D_TID (BID_X * NUM_THREAD_IN_BLOCK) + TID_IN_BLOCK
#define GRID_2D_TID (BID_Y * (Gdim_X * NUM_THREAD_IN_BLOCK) + GRID_1D_TID)
#define GLOBAL_TID (BID_Z * (Gdim_Y * Gdim_X * NUM_THREAD_IN_BLOCK) + GRID_2D_TID)

#endif

// 나중에 필요시 #include CUDA_definitions.cuh 추가하면 됨(같은 디렉토리에서)

In [ ]:
%%writefile MatAdd_G2D_B2D.cu
// 2차원 그리드, 2차원 블록 스레드 레이아웃에서 크기가 1024이상인 대규모 행렬 합
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
//#include "CUDA_definitions.cuh"

#include <stdio.h>
#include <stdlib.h>
#include <math.h>

__global__ void MatAdd_G2D_B2D
(float* MatA, float* MatB, float* MatC, int ROW_SIZE, int COL_SIZE)
{
    unsigned int col = threadIdx.x + blockIdx.x * blockDim.x;
    unsigned int row = threadIdx.y + blockIdx.y * blockDim.y;
    unsigned int index = row * COL_SIZE + col;

    if (col < COL_SIZE && row < ROW_SIZE) MatC[index] = MatA[index] + MatB[index];
}

int main() {
    dim3 blockDim(32, 32); // 블럭당 최대 스레드 개수 1024
    dim3 gridDim(ceil((float)COL_SIZE / blockDim.x), ceil((float)ROW_SIZE / blockDim.y));
    MatAdd_G2D_B2D <<< gridDim, blockDim >>> (A, B, C, ROW_SIZE, COL_SIZE);
    cudaDeviceSynchronize();
    // A, B, C는 행렬 배열 포인터, ROW_SIZE, COL_SIZE는 전처리된 사이즈
    return 0;
}


In [ ]:
%%writefile MatAdd_G1D_B1D.cu
// 1차원 그리드, 1차원 블록 스레드 레이아웃에서 크기가 1024이상인 대규모 행렬 합
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
//#include "CUDA_definitions.cuh"

#include <stdio.h>
#include <stdlib.h>
#include <math.h>

__global__ void MatAdd_G1D_B1D
(float* MatA, float* MatB, float* MatC, int ROW_SIZE, int COL_SIZE)
{
    unsigned int col = threadIdx.x + blockIdx.x * blockDim.x;
   if (col < COL_SIZE) {
      for (int row = 0; row < ROW_SIZE; row++) {
          int index = row * COL_SIZE + col;
          MatC[index] = MatA[index] + MatB[index];
      }
   }
}

int main() {
    dim3 blockDim(32); // 블럭당 최대 스레드 개수 1024
    dim3 gridDim(ceil((float)COL_SIZE / blockDim.x));
    MatAdd_G1D_B1D <<< gridDim, blockDim >>> (A, B, C, ROW_SIZE, COL_SIZE);
    cudaDeviceSynchronize();

    return 0;
}


In [ ]:
%%writefile MatAdd_G2D_B1D.cu
// 2차원 그리드, 1차원 블록 스레드 레이아웃에서 크기가 1024이상인 대규모 행렬 합
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
//#include "CUDA_definitions.cuh"

#include <stdio.h>
#include <stdlib.h>
#include <math.h>

__global__ void MatAdd_G2D_B1D
(float* MatA, float* MatB, float* MatC, int ROW_SIZE, int COL_SIZE)
{
    unsigned int col = threadIdx.x + blockIdx.x * blockDim.x;
    unsigned int row = blockIdx.y;
    unsigned int index = row * COL_SIZE + col;

    if (col < COL_SIZE && row < ROW_SIZE) MatC[index] = MatA[index] + MatB[index];

}

int main() {
    dim3 blockDim(32); // 블럭당 최대 스레드 개수 1024
    dim3 gridDim(ceil((float)COL_SIZE / blockDim.x), ROW_SIZE);
    MatAdd_G2D_B1D <<< gridDim, blockDim >>> (A, B, C, ROW_SIZE, COL_SIZE);
    cudaDeviceSynchronize();

    return 0;
}

In [ ]:
%%writefile Device_Query.cu

#include "cuda_runtime.h"
#include "device_launch_parameters.h"
//#include "helper_cuda.h"

#include <stdio.h>

#define _1MB (1024*1024)


int main() {
    int ngpus; // number of gpus
    cudaGetDeviceCount(&ngpus);
    for (int i = 0; i < ngpus; i++) {
        cudaDeviceProp devProp;

        cudaGetDeviceProperties(&devProp, i);

        printf("Device %d: %s\n", i, devProp.name);
        printf("\tCompute capability: %d.%d\n", devProp.major, devProp.minor);
        printf("\tThe number of streaming multiprocessors: %d\n", devProp.multiProcessorCount);
        //printf("\tThe number of CUDA cores: %d\n", _ConvertSMVer2Cores(devProp.major, devProp.minor) * devProp.multiProcessorCount);
        printf("\tGlobal memory size: %.2f MB", (float)devProp.totalGlobalMem / _1MB);
    }

    return 0;
}

Overwriting Device_Query.cu


In [ ]:
!nvcc -o Device_Query Device_Query.cu
!./Device_Query

Device 0: Tesla T4
	Compute capability: 7.5
	The number of streaming multiprocessors: 40
	Global memory size: 15102.06 MB

In [ ]:
%%writefile Matrix_Multiplcation.cu

#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>

#define BLOCK_SIZE 16

__global__ void matMul_kernel_lowerThan1024ver
(int* A, int* B, int* C, int m, int n, int k)
// A, B, C는 행렬 포인터, A의 차원: m * k, B의 차원: k * n, C의 차원: m * n
{
    int row = threadIdx.x;
    int col = threadIdx.y;
    int index = row * n + col;

    if (row >= m || col >= n) return;

    C[index] = 0;
    for (int offest = 0; offset < k; offset++) {
        C[index] += A[row * k + offset] * B[col + offset * n];
    }
}

__global__ void matMul_kernel_higherThan1024ver
(int* A, int* B, int* C, int m, int n, int k)
// A, B, C는 행렬 포인터, A의 차원: m * k, B의 차원: k * n, C의 차원: m * n
{
    int row = (blockDim.x * blockIdx.x) + threadIdx.x;
    int col = (blockDim.y * blockIdx.y) + threadIdx.y;
    int index = row * n + col;

    if (row >= m || col >= n) return;

    C[index] = 0;
    for (int offest = 0; offset < k; offset++) {
        C[index] += A[row * k + offset] * B[col + offset * n];
    }
}

int main(int argc, char* argv[]) {

    int m,n,k;
    m = atoi(argv[1]); n = atoi(argv[2]); k = atoi(argv[3]);

    int sizeA = m * k;
    int sizeB = k * n;
    int sizeC = m * n;

    int* dA, *dB, *dC;

    // 1. Allocate device memory for dA, dB dC
    cudaMalloc(&dA, sizeA * sizeof(int)); cudaMemset(dA, 0, sizeA * sizeof(int));
    cudaMalloc(&dB, sizeB * sizeof(int)); cudaMemset(dB, 0, sizeB * sizeof(int));
    cudaMalloc(&dC, sizeC * sizeof(int)); cudaMemset(dC, 0, sizeC * sizeof(int));

    // 2. Send(Copy) tje input matrices to GPU (A -> dA, B -> dB)
    cudaMemcpy(dA, A, sizeA * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dB, B, sizeB * sizeof(int), cudaMemcpyHostToDevice);

    // 3. Set the thread layout
    dim3 gridDim(ceil((float)m / BLOCK_SIZE), ceil((float)n / BLOCK_SIZE));
    dim3 blockDim(BLOCK_SIZE, BLOCK_SIZE);

    // 4. kernel call
    matMul_kernel_higherThan1024ver <<< gridDim, blockDim >>> (dA, dB, dC, m, n, k);
    cudaDeviceSynchronize();

    // 5. Get(Copy) the result from GPU to host memory (dC  -> Cgpu)
    cudaMemcpy(Cgpu, dC, sizeC * sizeof(int), cudaMemcpyDeviceToHost);

    // 6. Release device memory space
    cudaFree(dA);
    cudaFree(dB);
    cudaFree(dC);

    return 0;
}

In [ ]:
%%writefile Matrix_Multiplcation_Shared.cu
// 1024개의 스레드로 한 개 블록 이내의 경우 공유 메모리를 활용한 행렬 곱셈 예시
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <stdlib.h>
#include <math.h>

#define BLOCK_SIZE 16
#define ROW_SIZE 32
#define COL_SIZE 32
#define K_SIZE 128

__global__ void matMul_kernel_shared
(float* _A, float* _B, float* _C)
// _A, _B, _C는 행렬 포인터, A 차원은 32 * 128, B 차원은 128 * 32, C 차원은 32 * 32
{
    int row = threadIdx.x;
    int col = threadIdx.y;
    int index = row * blockDim.y + col;

    __shared__ float sA[ROW_SIZE][K_SIZE]; // 23*256*4 bytes = 16KB
    __shared__ float sB[K_SIZE][COL_SIZE]; // 16KB
    // 합계 32kb 는 48KB~96KB (GPU 공유 메모리)보다 작으므로 가능(공유 메모리 정적 할당)

    if (row == 0) { // 첫 row 스레드가 read matrix B의 column을 다 공유 메모리에 넣음
        for (int k = 0; k < K_SIZE; k++) sB[k][col] = _B[col + k * COL_SIZE];
    }

    if (col == 0) { // 첫 col 스레드가 read matrix A의 row을 다 공유 메모리에 넣음
        for (int k = 0; k < K_SIZE; k++) sA[row][k] = _B[row * K_SIZE + k];
    }

    __syncthreads(); // wait until all threads load the matrix

    float result = 0;
    for (int k = 0; k < K_SIZE; k++) result += sA[row][k] * sB[k][col];
    _C[index] = result;
}


int main(int argc, char* argv[]) {

    int sizeA = ROW_SIZE * K_SIZE;
    int sizeB = K_SIZE * COL_SIZE;
    int sizeC = ROW_SIZE * COL_SIZE;

    int* dA, *dB, *dC;

    // 1. Allocate device memory for dA, dB dC
    cudaMalloc(&dA, sizeA * sizeof(int)); cudaMemset(dA, 0, sizeA * sizeof(int));
    cudaMalloc(&dB, sizeB * sizeof(int)); cudaMemset(dB, 0, sizeB * sizeof(int));
    cudaMalloc(&dC, sizeC * sizeof(int)); cudaMemset(dC, 0, sizeC * sizeof(int));

    // 2. Send(Copy) tje input matrices to GPU (A -> dA, B -> dB)
    cudaMemcpy(dA, A, sizeA * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dB, B, sizeB * sizeof(int), cudaMemcpyHostToDevice);

    // 3. Set the thread layout
    dim3 gridDim(ceil((float)ROW_SIZE / BLOCK_SIZE), ceil((float)COL_SIZE / BLOCK_SIZE));
    dim3 blockDim(BLOCK_SIZE, BLOCK_SIZE);

    // 4. kernel call
    matMul_kernel_shared <<< gridDim, blockDim >>> (dA, dB, dC);
    cudaDeviceSynchronize();

    // 5. Get(Copy) the result from GPU to host memory (dC  -> C)
    cudaMemcpy(C, dC, sizeC * sizeof(int), cudaMemcpyDeviceToHost);

    // 6. Release device memory space
    cudaFree(dA);
    cudaFree(dB);
    cudaFree(dC);

    return 0;
}

In [ ]:
%%writefile Matrix_Multiplcation_Shared_Large.cu

#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>

#define BLOCK_SIZE 16

__global__ void matMul_kernel_shared_large
(int* matA, int* matB, int* matC, int m, int n, int k)
// A, B, C는 행렬 포인터, A의 차원: m * k, B의 차원: k * n, C의 차원: m * n
{
    int row = (blockDim.x * blockIdx.x) + threadIdx.x;
    int col = (blockDim.y * blockIdx.y) + threadIdx.y;

    int val = 0;
    __shared__ int subA[BLOCK_SIZE][BLOCK_SIZE];
    __shared__ int subB[BLOCK_SIZE][BLOCK_SIZE];

    int localRow = threadIdx.x;
    int localCol = threadIdx.y;

    for (int bID = 0; bID < ceil((float)k / BLOCK_SIZE); bID++) {
        int stride = bId * BLOCK_SIZE;

        if (row >= m || stride + localCol >= k) subA[localRow][localCol] = 0;
        else subA[localRow][localCol] = matA[row * k + (stride + localCol)];

        if (col >= n || stride + localRow >= k) subB[localRow][localCol] = 0;
        else subB[localRow][localCol] = matB[(stride + localRow) * n + col];

        __syncthreads(); // 모든 데이터의 복사가 완료될 때까지 대기

        for (int i = 0; i < BLOCK_SIZE; i++) {
            val += subA[localRow][i] * subB[i][localCol];
        } // 서브 블록 행렬 계산 (C(localRow, localCol))
        __syncthreads(); // 모든 스레드 계산 완료 대기
    }

    if (row >= m || col >= n) return;

    matC[row * n + col] = val;
}

int main(int argc, char* argv[]) {

    int m,n,k; // 1024로 시도하기
    m = atoi(argv[1]); n = atoi(argv[2]); k = atoi(argv[3]);

    int sizeA = m * k;
    int sizeB = k * n;
    int sizeC = m * n;

    int* dA, *dB, *dC;

    // 1. Allocate device memory for dA, dB dC
    cudaMalloc(&dA, sizeA * sizeof(int)); cudaMemset(dA, 0, sizeA * sizeof(int));
    cudaMalloc(&dB, sizeB * sizeof(int)); cudaMemset(dB, 0, sizeB * sizeof(int));
    cudaMalloc(&dC, sizeC * sizeof(int)); cudaMemset(dC, 0, sizeC * sizeof(int));

    // 2. Send(Copy) tje input matrices to GPU (A -> dA, B -> dB)
    cudaMemcpy(dA, A, sizeA * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dB, B, sizeB * sizeof(int), cudaMemcpyHostToDevice);

    // 3. Set the thread layout
    dim3 gridDim(ceil((float)m / BLOCK_SIZE), ceil((float)n / BLOCK_SIZE));
    dim3 blockDim(BLOCK_SIZE, BLOCK_SIZE);

    // 4. kernel call
    matMul_kernel_shared_large <<< gridDim, blockDim >>> (dA, dB, dC, m, n, k);
    cudaDeviceSynchronize();

    // 5. Get(Copy) the result from GPU to host memory (dC  -> C)
    cudaMemcpy(C, dC, sizeC * sizeof(int), cudaMemcpyDeviceToHost);

    // 6. Release device memory space
    cudaFree(dA);
    cudaFree(dB);
    cudaFree(dC);

    return 0;
}

In [1]:
%%writefile WarpSynchronization.cu

#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>

#define BLOCK_SIZE 64

__global__ void syncWarp_test()
{
    int tID = threadIdx.x;
    int warpID = (int) (tID / 32);
    __shared__ int masterID[BLOCK_SIZE/32];

    if (threadIdx.x % 32 == 0) {
        masterID[warpID] = tID;
    }
    __syncwarp(); // intra-warp synchronization (barrier)

    printf("[T%d] The master of our warp is %d\n", tID, masterID [warpID]);
}

int main() {
    syncWarp_test <<< 1, BLOCK_SIZE >>> ();
    cudaDeviceSynchronize();  // Ensure the kernel completes before the program exits

    cudaError_t err = cudaGetLastError();  // Check for kernel launch errors
    if (err != cudaSuccess) {
        printf("CUDA Error: %s\n", cudaGetErrorString(err));
    }

    return 0;
}

Writing WarpSynchronization.cu


In [2]:
!nvcc -o WarpSynchronization WarpSynchronization.cu
!./WarpSynchronization

[T0] The master of our warp is 0
[T1] The master of our warp is 0
[T2] The master of our warp is 0
[T3] The master of our warp is 0
[T4] The master of our warp is 0
[T5] The master of our warp is 0
[T6] The master of our warp is 0
[T7] The master of our warp is 0
[T8] The master of our warp is 0
[T9] The master of our warp is 0
[T10] The master of our warp is 0
[T11] The master of our warp is 0
[T12] The master of our warp is 0
[T13] The master of our warp is 0
[T14] The master of our warp is 0
[T15] The master of our warp is 0
[T16] The master of our warp is 0
[T17] The master of our warp is 0
[T18] The master of our warp is 0
[T19] The master of our warp is 0
[T20] The master of our warp is 0
[T21] The master of our warp is 0
[T22] The master of our warp is 0
[T23] The master of our warp is 0
[T24] The master of our warp is 0
[T25] The master of our warp is 0
[T26] The master of our warp is 0
[T27] The master of our warp is 0
[T28] The master of our warp is 0
[T29] The master of our 

In [3]:
%%writefile atomicAdd.cu

#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <iostream>
#include <chrono>

#define GRID_SIZE 128 * 4
#define BLOCK_SIZE 1024

__global__ void threadCounting_noSync(int *a)
{
    (*a)++;
}

__global__ void threadCounting_atomicGlobal(int *a)
{
    atomicAdd(a, 1);
}

__global__ void threadCounting_atomicShared(int *a)
{
    __shared__ int sa; // 블록 당 할당된 공유 메모리
    if (threadIdx.x == 0) sa = 0;
    // 대표 스레드에서 공유 메모리 초기화
    __syncthreads(); // barrier for all initialization

    atomicAdd(&sa, 1); //block-level counting

    __syncthreads(); // barrier for all operations

    if (threadIdx.x == 0) atomicAdd(a, sa);
    // grid-level counting
    // 각 블록에서 하나의 스레드만 원자 함수를 호출하므로
    // 동기화 참여 스레드 수는 블록의 수와 같음
}

int main() {
    int *noSyncKernels;
    int host_noSync = 0;

    cudaMalloc(&noSyncKernels, sizeof(int));
    cudaMemset(noSyncKernels, 0, sizeof(int));

    auto start_nosync = std::chrono::high_resolution_clock::now();

    threadCounting_noSync <<< GRID_SIZE, BLOCK_SIZE >>> (noSyncKernels);
    cudaDeviceSynchronize();  // Ensure the kernel completes before the program exits

    auto end_nosync = std::chrono::high_resolution_clock::now();

    cudaError_t err = cudaGetLastError();  // Check for kernel launch errors
    if (err != cudaSuccess) {
        printf("CUDA Error: %s\n", cudaGetErrorString(err));
        return 1;
    }

    cudaMemcpy(&host_noSync, noSyncKernels, sizeof(int), cudaMemcpyDeviceToHost);

    std::chrono::duration<double> elapsed_nosync = end_nosync - start_nosync;

    printf("[No sync] # of threads = %d\n", host_noSync);
    std::cout << "No Sync Elapsed time: " << elapsed_nosync.count() * 1000 << " ms" << std::endl;

    cudaFree(noSyncKernels);

    ///////////////////////////////////////////////////////////////////

    int *SyncKernels;
    int host_Sync = 0;

    cudaMalloc(&SyncKernels, sizeof(int));
    cudaMemset(SyncKernels, 0, sizeof(int));

    auto start_sync = std::chrono::high_resolution_clock::now();

    threadCounting_atomicGlobal <<< GRID_SIZE, BLOCK_SIZE >>> (SyncKernels);
    cudaDeviceSynchronize();  // Ensure the kernel completes before the program exits

    auto end_sync = std::chrono::high_resolution_clock::now();

    cudaError_t err2 = cudaGetLastError();  // Check for kernel launch errors
    if (err2 != cudaSuccess) {
        printf("CUDA Error: %s\n", cudaGetErrorString(err2));
        return 1;
    }

    cudaMemcpy(&host_Sync, SyncKernels, sizeof(int), cudaMemcpyDeviceToHost);

    std::chrono::duration<double> elapsed_sync = end_sync - start_sync;

    printf("[Atomic Global] # of threads = %d\n", host_Sync);
    std::cout << "Atomic Elapsed time: " << elapsed_sync.count() * 1000 << " ms" << std::endl;

    cudaFree(SyncKernels);

    //////////////////////////////////////////////////////////////////

    int *SyncSharedKernels;
    int host_Sync_Shared = 0;

    cudaMalloc(&SyncSharedKernels, sizeof(int));
    cudaMemset(SyncSharedKernels, 0, sizeof(int));

    auto start_sync_shared = std::chrono::high_resolution_clock::now();

    threadCounting_atomicGlobal <<< GRID_SIZE, BLOCK_SIZE >>> (SyncSharedKernels);
    cudaDeviceSynchronize();  // Ensure the kernel completes before the program exits

    auto end_sync_shared = std::chrono::high_resolution_clock::now();

    cudaError_t err3 = cudaGetLastError();  // Check for kernel launch errors
    if (err3 != cudaSuccess) {
        printf("CUDA Error: %s\n", cudaGetErrorString(err3));
        return 1;
    }

    cudaMemcpy(&host_Sync_Shared, SyncSharedKernels, sizeof(int), cudaMemcpyDeviceToHost);

    std::chrono::duration<double> elapsed_sync_shared = end_sync_shared - start_sync_shared;

    printf("[Atomic Shared] # of threads = %d\n", host_Sync_Shared);
    std::cout << "Shared Atomic Elapsed time: " << elapsed_sync_shared.count() * 1000 << " ms" << std::endl;

    cudaFree(SyncSharedKernels);

    return 0;
}

Overwriting atomicAdd.cu


In [4]:
!nvcc -o atomicAdd atomicAdd.cu
!./atomicAdd

[No sync] # of threads = 25
No Sync Elapsed time: 0.149171 seconds
[Atomic Global] # of threads = 524288
Atomic Elapsed time: 6.9282e-05 seconds
[Atomic Shared] # of threads = 524288
Shared Atomic Elapsed time: 4.0275e-05 seconds


In [3]:
%%writefile MultiStreamAsync.cu

#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <iostream>
#include <chrono>

#define NUM_BLOCK (128 * 1024)
#define ARRAY_SIZE (1024 * NUM_BLOCK)
#define NUM_STREAMS 4
#define WORK_LOAD 256

__global__ void myKernel(int *_in, int* _out)
{
    int tID = blockDim.x * blockDim.x + threadIdx.x;

    int temp = 0;
    int in = _in[tID];
    for (int i = 0; i < WORK_LOAD; i++) {
        temp = (temp + in % 5) % 10;
    }
    _out[tID] = temp;
} // 단순 내부 연산 시키기

int main() {

    int *in = NULL, *out = NULL, *dIn = NULL, *dOut = NULL;

    cudaMallocHost(&in, sizeof(int) * ARRAY_SIZE); // pinned memory
    memset(in, 0, sizeof(int) * ARRAY_SIZE); // host 메모리 초기화

    cudaMallocHost(&out, sizeof(int) * ARRAY_SIZE); // pinned memory
    memset(out, 0, sizeof(int) * ARRAY_SIZE); // host 메모리 초기화

    cudaMalloc(&dIn, sizeof(int) * ARRAY_SIZE);
    cudaMalloc(&dOut, sizeof(int) * ARRAY_SIZE); // gpu global memoey 할당

    for (int i = 0; i < ARRAY_SIZE; i++) in[i] = rand() % 10; // 배열 초기화

    // single stream version (동기)
    cudaMemcpy(dIn, in, sizeof(int) * ARRAY_SIZE, cudaMemcpyHostToDevice);
    myKernel <<< NUM_BLOCK, 1024 >>> (dIn, dOut);
    cudaMemcpy(out, dOut, sizeof(int) * ARRAY_SIZE, cudaMemcpyDeviceToHost);

    // multi-stream version
    cudaStream_t stream[NUM_STREAMS]; // Non-Null 스트림 변수 선언
    for (int i = 0; i < NUM_STREAMS; i++) cudaStreamCreate(&stream[i]); // 스트림 생성

    int chunkSize = ARRAY_SIZE / NUM_STREAMS; // 스트림 당 데이터

    for (int i = 0; i < NUM_STREAMS; i++) {
        int offset = chunkSize * i;
        cudaMemcpyAsync(dIn + offset, in + offset, sizeof(int) * chunkSize,
            cudaMemcpyHostToDevice, stream[i]);
    } // 각 스트림 당 호스트에서 디바이스로 메모리 옮김

    for (int i = 0; i < NUM_STREAMS; i++) {
        int offset = chunkSize * i;
        myKernel <<< NUM_BLOCK / NUM_STREAMS, 1024, 0, stream[i] >>>
            (dIn + offset, dOut + offset);
    } // 각 스트림 당 해당 메모리 영역에 대해 커널 수행

    for (int i = 0; i < NUM_STREAMS; i++) {
        int offset = chunkSize * i;
        cudaMemcpyAsync(out + offset, dOut + offset, sizeof(int) * chunkSize,
            cudaMemcpyDeviceToHost, stream[i]);
    } // 각 스트림 당 디바이스에서 호스트로 결과 메모리 옮김

    cudaDeviceSynchronize();

    for (int i = 0; i < NUM_STREAMS; i++) cudaStreamDestroy(stream[i]); // 스트림 제거

    cudaFree(dIn);
    cudaFree(dOut); // gpu 메모리 할당 해제
    cudaFreeHost(in);
    cudaFreeHost(out); // pinned memory 해제

    return 0;
}


Overwriting MultiStreamAsync.cu


In [4]:
!nvcc -o MultiStreamAsync MultiStreamAsync.cu
!./MultiStreamAsync

In [3]:
%%writefile StreamEvent.cu
// 비동기로 동작 가능한 멀티 스트림을 cuda event로 수행 시간을 측정한다.
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <iostream>
#include <chrono>

#define NUM_BLOCK (128 * 1024)
#define ARRAY_SIZE (1024 * NUM_BLOCK)
#define NUM_STREAMS 4
#define WORK_LOAD 256

__global__ void myKernel(int *_in, int* _out)
{
    int tID = blockDim.x * blockDim.x + threadIdx.x;

    int temp = 0;
    int in = _in[tID];
    for (int i = 0; i < WORK_LOAD; i++) {
        temp = (temp + in % 5) % 10;
    }
    _out[tID] = temp;
} // 단순 내부 연산 시키기

int main() {

    int *in = NULL, *out = NULL, *dIn = NULL, *dOut = NULL;

    cudaMallocHost(&in, sizeof(int) * ARRAY_SIZE); // pinned memory
    memset(in, 0, sizeof(int) * ARRAY_SIZE); // host 메모리 초기화

    cudaMallocHost(&out, sizeof(int) * ARRAY_SIZE); // pinned memory
    memset(out, 0, sizeof(int) * ARRAY_SIZE); // host 메모리 초기화

    cudaMalloc(&dIn, sizeof(int) * ARRAY_SIZE);
    cudaMalloc(&dOut, sizeof(int) * ARRAY_SIZE); // gpu global memoey 할당

    for (int i = 0; i < ARRAY_SIZE; i++) in[i] = rand() % 10; // 배열 초기화

    // multi-stream version
    cudaStream_t stream[NUM_STREAMS]; // Non-Null 스트림 변수 선언 (배열)
    cudaEvent_t start[NUM_STREAMS], end[NUM_STREAMS]; // cuda 이벤트 변수 선언 (이벤트 배열)

    for (int i = 0; i < NUM_STREAMS; i++) {
        cudaStreamCreate(&stream[i]); // 스트림 생성
        cudaEventCreate(&start[i]); cudaEventCreate(&end[i]); // 이벤트 생성
    }

    int chunkSize = ARRAY_SIZE / NUM_STREAMS; // 스트림 당 데이터

    int offset[NUM_STREAMS] = {0};
    for (int i = 0; i < NUM_STREAMS; i++) offset[i] = chunkSize * i;

    for (int i = 0; i < NUM_STREAMS; i++) {
        cudaEventRecord(start[i], stream[i]); // cuda event 기록: cuda event를 stream에 넣음

        cudaMemcpyAsync(dIn + offset[i], in + offset[i], sizeof(int) * chunkSize,
            cudaMemcpyHostToDevice, stream[i]);
    } // 각 스트림 당 호스트에서 디바이스로 메모리 옮김

    for (int i = 0; i < NUM_STREAMS; i++) {
        myKernel <<< NUM_BLOCK / NUM_STREAMS, 1024, 0, stream[i] >>>
            (dIn + offset[i], dOut + offset[i]);
    } // 각 스트림 당 해당 메모리 영역에 대해 커널 수행

    for (int i = 0; i < NUM_STREAMS; i++) {
        cudaMemcpyAsync(out + offset[i], dOut + offset[i], sizeof(int) * chunkSize,
            cudaMemcpyDeviceToHost, stream[i]);

        cudaEventRecord(end[i], stream[i]); // 각 스트림 당 end event를 넣음
    } // 각 스트림 당 디바이스에서 호스트로 결과 메모리 옮김

    cudaDeviceSynchronize();

    for (int i = 0; i < NUM_STREAMS; i++) {
        if (cudaEventQuery(start[i]) == cudaSuccess
            && cudaEventQuery(end[i]) == cudaSuccess) {
                float time = 0;
                cudaEventElapsedTime(&time, start[i], end[i]);
                printf("Stream[%d] : %f ms\n", i, time);
            } // 이벤트가 성공적으로 일어났다면
        else {
            printf("Event has not occured!");
        }
    }

    for (int i = 0; i < NUM_STREAMS; i++) {
        cudaStreamDestroy(stream[i]); // 스트림 제거
        cudaEventDestroy(start[i]); cudaEventDestroy(end[i]); // 이벤트 제거
    }

    cudaFree(dIn);
    cudaFree(dOut); // gpu 메모리 할당 해제
    cudaFreeHost(in);
    cudaFreeHost(out); // pinned memory 해제

    return 0;
}


Overwriting StreamEvent.cu


In [4]:
!nvcc -o StreamEvent StreamEvent.cu
!./StreamEvent

Stream[0] : 147.220383 ms
Stream[1] : 169.960220 ms
Stream[2] : 192.696701 ms
Stream[3] : 215.471710 ms
